# <span style="color:#ff5f27"> 👨🏻‍🏫 Vehicle Sales Price Predictions Workshop - Part 3 of 3</span>

## <span style="color:#ff5f27"> 🚀 Inference Pipeline </span>

In order to make a machine learning system from this dataset, we have structured the service into 3 pipelines:

1. feature engineering pipeline notebook (see Part 1)
2. training pipeline notebook (see Part 2)
3. inferencing pipeline notebook (this Part 3)

This notebook will outline the third step, ie. the inference pipeline.

## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
!pip install --quiet gradio==3.48.0

In [ ]:
import joblib
import pandas as pd
from functools import partial
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    "car_prices", 
    version=1,
)

## <span style="color:#ff5f27"> 🗄 Model Registry </span>

In [ ]:
mr = project.get_model_registry()

model = mr.get_model(
    "car_prices",
    version=1,
)

# Download the model directory from the Model Registry
model_dir = model.download()

# Load the model using joblib from the downloaded model directory
label_encoders = joblib.load(model_dir + "/label_encoders.pkl")

## <span style="color:#ff5f27"> 🚀 Fetch Deployment </span>

In [ ]:
# Access the Model Serving
ms = project.get_model_serving()

# Specify the deployment name
deployment_name = "carpricemodeldeployment"

# Get the deployment with the specified name
deployment = ms.get_deployment(deployment_name)

# Start the deployment and wait for it to be in a running state for up to 300 seconds
deployment.start(await_running=300)

## <span style="color:#ff5f27">🔮 Inference using Model that has been Deploymed on Hopsworks </span>

In [ ]:
# Define the function to encode categorical data
def encode_categorical_data(dataset, label_encoders):
    """
    Encodes categorical features in the dataset using the provided label encoders.

    Parameters:
    - dataset: DataFrame
        The input data containing the features for prediction.
    - label_encoders: dict
        A dictionary of pre-fitted label encoders for categorical features.

    Returns:
    - DataFrame
        The processed input data with categorical features encoded.
    """
    # Iterate over the columns of the DataFrame
    for column in dataset.columns:
        # Check if the column is of type 'object' (categorical)
        if dataset[column].dtype == 'object':
            # Retrieve already fitted LabelEncoder
            label_encoder = label_encoders[column.lower()]
            # Perform encoding on unique column values
            dataset[column] = label_encoder.transform(dataset[column])
    return dataset

In [ ]:
def predict_selling_price(input_data, label_encoders, deployment):
    """
    Predicts the selling price of a car based on input features.

    Parameters:
    - input_data: DataFrame
        The input data containing the features for prediction.
    - label_encoders: dict
        A dictionary of pre-fitted label encoders for categorical features.
    - deployment: object
        The deployment object that has the predict method.

    Returns:
    - float
        The predicted selling price, rounded to two decimal places.
    """
    # Encode categorical data using label encoders
    processed_data = encode_categorical_data(input_data, label_encoders)
    
    # Predict the selling price using the deployment's predict method
    prediction = deployment.predict(
        inputs=list(processed_data.values[0])  # Convert processed data to a list of values
    )['predictions']  # Extract the predictions from the response
    # Return the first prediction rounded to two decimal places
    return round(prediction[0], 2)

In [ ]:
# Define the method to print out the values entered
def print_values(year, make, model, trim, body, transmission, condition, odometer, color, interior, label_encoders, deployment):
    """
    Collects input values, encodes categorical data, and predicts the selling price.

    Parameters:
    - year, make, model, trim, body, transmission, condition, odometer, color, interior: various
        The input features for the prediction.
    - label_encoders: dict
        A dictionary of pre-fitted label encoders for categorical features.
    - deployment: object
        The deployment object that has the predict method.

    Returns:
    - float
        The predicted selling price, rounded to two decimal places.
    """
    # Create a dictionary with the input data
    data = {
        "Year": [year],
        "Make": [make],
        "Model": [model],
        "Trim": [trim],
        "Body": [body],
        "Transmission": [transmission],
        "Condition": [condition],
        "Odometer": [odometer],
        "Color": [color],
        "Interior": [interior]
    }
    # Convert the dictionary to a DataFrame
    df = pd.DataFrame(data)
    
    # Predict the selling price
    prediction = predict_selling_price(df, label_encoders, deployment)
    return prediction


# Partial function to include label_encoders and deployment
print_values_partial = partial(
    print_values, 
    label_encoders=label_encoders,
    deployment=deployment,
)

## <span style="color:#ff5f27">🎮 Gradio Interface as Alternative to Deployment-based prediction</span>

In [ ]:
# Create the Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# 🚗 Car Price Prediction")
    
    with gr.Row():
        year = gr.Number(label="Year", value=2014)
        make = gr.Dropdown(label="Make", choices=["Toyota", "Ford", "Volkswagen", "BMW"], value="Toyota")
        model = gr.Dropdown(label="Model", choices=["Prius", "Mustang", "Jetta", "X1"], value="Prius")
        trim = gr.Dropdown(label="Trim", choices=["Base", "SL", "GLS", "Luxury"], value="Base")
        body = gr.Dropdown(label="Body", choices=["Hatchback", "Sedan", "SUV", "Minivan"], value="Hatchback")

    with gr.Row():
        transmission = gr.Dropdown(label="Transmission", choices=["automatic", "manual"], value="automatic")
        condition = gr.Number(label="Condition", value=45.0)
        odometer = gr.Number(label="Odometer", value=33761.0)
        color = gr.Dropdown(label="Color", choices=["red", "white", "black", "silver"], value="red")
        interior = gr.Dropdown(label="Interior", choices=["black", "gray", "brown"], value="black")

    submit_button = gr.Button("Submit")
    output = gr.JSON(label="Entered Values")

    submit_button.click(
        print_values_partial,
        inputs=[year, make, model, trim, body, transmission, condition, odometer, color, interior],
        outputs=output,
    )

# Launch the interface
demo.launch(share=True)

This completes the entire process of feature engineering, training and inferencing pipelines, and therefore the delivery of an end-to-end machine learning system.

---